In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle

In [13]:
# Charger les données
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_data_dict = pd.read_excel('data dictionary.xlsx')

# Définir les colonnes à sélectionner pour l'entraînement du modèle
features = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Fuel_Type', 'Transmission', 'Owner_Type']


In [15]:
# Vectoriser la colonne "Name" en utilisant TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train['Name'])

# Appliquer le clustering k-means
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)

# Ajouter les étiquettes de cluster à la colonne "Name"
df_train['Name_Cluster'] = kmeans.labels_

# Attribuer les noms de clusters les plus fréquents
cluster_names = []
for i in range(10):
    names = df_train['Name'][df_train['Name_Cluster'] == i]
    counter = Counter(names)
    cluster_name = counter.most_common(1)[0][0]
    cluster_names.append(cluster_name)

# Remplacer les valeurs dans la colonne "Name_Cluster" par les nouveaux noms de cluster
cluster_map = dict(zip(range(10), cluster_names))
df_train['Name_Cluster'] = df_train['Name_Cluster'].map(cluster_map)


In [16]:
# Nettoyer et préparer les données
df_train['Engine'] = df_train['Engine'].str.replace(' CC', '', regex=False)
df_train['Mileage'] = df_train['Mileage'].str.replace(' kmpl| km/kg', '', regex=True).astype(float)
df_train['Power'] = df_train['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)
df_train = df_train.drop(['New_Price'], axis=1)
df_test = df_test.drop(['New_Price'], axis=1)

num_cols = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats']
cat_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Name_Cluster']

# Séparer les données en features et cible
X = df_train.drop(['Price'], axis=1)
y = df_train['Price']

In [17]:
# Définir les transformations pour les colonnes numériques et catégorielles
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combiner les transformations en une seule étape de prétraitement
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Définir le modèle de régression
model = GradientBoostingRegressor(random_state=42)

# Créer la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [18]:
# Diviser les données d'entraînement en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle sur les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de validation
score = pipeline.score(X_val, y_val)
print('Score de la pipeline sur les données de validation :', score)

# Faire des prédictions sur les données de validation
y_pred = pipeline.predict(X_val)

# Coefficient de détermination (R²)
r2 = r2_score(y_val, y_pred)
print("R² :", r2)

# Erreur moyenne absolue (MAE)
mae = mean_absolute_error(y_val, y_pred)
print("MAE :", mae)

# Erreur quadratique moyenne (MSE)
mse = mean_squared_error(y_val, y_pred)
print("MSE :", mse)

# Erreur quadratique moyenne racine (RMSE)
rmse = np.sqrt(mse)
print("RMSE :", rmse)

Score de la pipeline sur les données de validation : 0.8577176866864733
R² : 0.8577176866864733
MAE : 1.9924380632134868
MSE : 17.509189181756298
RMSE : 4.184398305820838


In [19]:
# Enregistrer le modèle entraîné sous forme de fichier pickle
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)


In [20]:
# Nettoyer et préparer les données de test
df_test['Engine'] = df_test['Engine'].str.replace(' CC', '', regex=False)
df_test['Mileage'] = df_test['Mileage'].str.replace(' kmpl| km/kg', '', regex=True).astype(float)
df_test['Power'] = df_test['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

# Recréer le vectorizer pour la colonne 'Name'
vectorizer = TfidfVectorizer()
vectorizer.fit(df_train['Name'])

# Recréer le modèle KMeans pour la colonne 'Name'
kmeans = KMeans(n_clusters=10)
kmeans.fit(vectorizer.transform(df_train['Name']))

# Charger le modèle entraîné à partir du fichier pickle
with open('trained_model.pkl', 'rb') as file:
    loaded_pipeline = pickle.load(file)

# Transformer la colonne 'Name' dans df_test en utilisant le vectorizer
name_features = vectorizer.transform(df_test['Name'])

# Prédire les clusters pour la colonne 'Name' dans df_test
df_test['Name_Cluster'] = kmeans.predict(name_features)

# Séparer les données en features (la colonne Price n'est pas présente dans df_test)
X_test = df_test.copy()

# Transformer les données en utilisant le pipeline chargé
X_test_transformed = loaded_pipeline.named_steps['preprocessor'].transform(X_test)

# Faire des prédictions sur les données de test
y_pred = loaded_pipeline.named_steps['model'].predict(X_test_transformed)

# Ajouter les prédictions à la colonne 'New_Price' dans df_test
df_test['Price'] = y_pred

# Arrondir les valeurs de la colonne 'New_Price' à 2 décimales
df_test['Price'] = df_test['Price'].round(2)


In [21]:
df_test.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Name_Cluster,Price
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26,998,58.20,4.0,9,4.18
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.70,796,47.30,5.0,9,2.74
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68,2393,147.80,7.0,2,18.00
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59,1364,NaN,5.0,2,4.86
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.50,1197,82.85,5.0,1,4.58
5,Mahindra XUV500 W8 2WD,Coimbatore,2016,85609,Diesel,Manual,Second,16.00,2179,140.00,7.0,2,11.93
6,Toyota Fortuner 4x2 AT TRD Sportivo,Pune,2015,59000,Diesel,Automatic,First,12.55,2982,168.70,7.0,2,23.35
7,Hyundai EON Era Plus,Jaipur,2013,65000,Petrol,Manual,First,21.10,814,55.20,5.0,2,2.99
8,Honda City 1.5 S MT,Mumbai,2011,66000,Petrol,Manual,Second,17.00,1497,118.00,5.0,8,3.72
9,Mahindra XUV500 W6 2WD,Coimbatore,2015,54684,Diesel,Manual,First,15.10,2179,140.00,7.0,2,11.35


In [23]:
conversion_rate = 87
lakh_conversion = 100000

# Convertir les prix en unités de lakhs en roupies indiennes
df_test['Price_Rupie'] = df_test['Price'] * lakh_conversion

# Convertir les prix en euros
df_test['Price_EUR'] = df_test['Price_Rupie'] / conversion_rate

# Arrondir les valeurs de la colonne 'New_Price_EUR' à 2 décimales
df_test['Price_EUR'] = df_test['Price_EUR'].round(2)

df_test.head()



,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Name_Cluster,Price,Price_Rupie,Price_EUR
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26,998,58.20,4.0,9,4.18,418000.0,4804.60
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.70,796,47.30,5.0,9,2.74,274000.0,3149.43
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68,2393,147.80,7.0,2,18.00,1800000.0,20689.66
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59,1364,NaN,5.0,2,4.86,486000.0,5586.21
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.50,1197,82.85,5.0,1,4.58,458000.0,5264.37


In [24]:
#   si on veux utiliser une fonction pour definire le modele et changer les hyperparametres 





def create_gb_model(n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1.0, loss='ls'):

    model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
                                     min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                     subsample=subsample, loss=loss, random_state=42)
    return model
